In [7]:
public enum PivotType { Undefined, Low, High }

public record PriceInfo(DateTime Date,
    double Open,
    double High,
    double Low,
    double Close,
    double ChannelLow,
    double ChannelHigh,
    PivotType PivotType,
    double PivotValue);

In [8]:
using System.Globalization;
using System.IO;

List<PriceInfo> ParseCsv(string filePath)
{
    var records = new List<PriceInfo>();
    // Assuming the first line of the CSV contains headers and each subsequent line is a record.
    foreach (var line in File.ReadAllLines(filePath).Skip(1))
    {
        var columns = line.Split(',');

        var pivotType = string.IsNullOrWhiteSpace(columns[7]) 
            ? PivotType.Undefined 
            : Enum.Parse<PivotType>(columns[7]);

        double pivotValue = pivotType == PivotType.Undefined 
            ? 0 
            : double.Parse(columns[8]);
        var record = new PriceInfo
        (
            Date: DateTime.ParseExact(columns[0], "yyyy-MM-dd HH:mm:ss", CultureInfo.InvariantCulture),
            Open: double.Parse(columns[1]),
            High: double.Parse(columns[2]),
            Low: double.Parse(columns[3]),
            Close: double.Parse(columns[4]),
            ChannelLow: double.Parse(columns[5]),
            ChannelHigh: double.Parse(columns[6]),
            PivotType: pivotType,
            PivotValue: pivotValue
        );
        records.Add(record);
    }
    return records;
}

In [9]:
#r "nuget: XPlot.Plotly, 4.0.6"
using XPlot.Plotly;



var targetFile = Environment.CurrentDirectory + "/breakout.csv";
var records = ParseCsv(targetFile);

var candlestickData = new Candlestick
{
    x = records.Select(r => r.Date.ToString("yyyy-MM-dd HH:mm:ss")).ToArray(),
    open = records.Select(r => r.Open).ToArray(),
    high = records.Select(r => r.High).ToArray(),
    low = records.Select(r => r.Low).ToArray(),
    close = records.Select(r => r.Close).ToArray(),
    name = "Candlestick"
};

var pivotPointsHigh = records.Where(r => r.PivotType == PivotType.High).ToList();
var pivotPointsLow = records.Where(r => r.PivotType == PivotType.Low).ToList();

var pivotHigh = new Scatter
{
    x = pivotPointsHigh.Select(r => r.Date.ToString("yyyy-MM-dd HH:mm:ss")).ToArray(),
    y = pivotPointsHigh.Select(r => r.PivotValue).ToArray(),
    mode = "markers",
    marker = new Marker
    {
        color = "Green",
        size = 10
    },
    name = "Pivot High"
};

var pivotLow = new Scatter
{
    x = pivotPointsLow.Select(r => r.Date.ToString("yyyy-MM-dd HH:mm:ss")).ToArray(),
    y = pivotPointsLow.Select(r => r.PivotValue).ToArray(),
    mode = "markers",
    marker = new Marker
    {
        color = "Red",
        size = 10
    },
    name = "Pivot Low"
};

var chart = Chart.Plot(new Trace[] { candlestickData, pivotHigh, pivotLow });
chart.WithLayout(new Layout.Layout
{
    title = "Breakout Analysis",
    xaxis = new Xaxis { title = "Date" },
    yaxis = new Yaxis { title = "Price" }
});

chart.Show();





Installed Packages XPlot.Plotly, 4.0.6